In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1693685709301_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#import boto3
# import pandas as pd
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate, lit, regexp_replace
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def Freq(pTabela,pColuna):
 
    qtd_total=pTabela.count()
   
    pTabela.registerTempTable("tab_input")
    frq = spark.sql(    
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))
       
    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"
    
    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.appName('trusted_metadata').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucket_control = "bkt-musicstream-bi/Files/Control"
bucket_ingestion = "bkt-musicstream-bi/Files/TransientZone"
bucket_raw = "bkt-musicstream-bi/Files/RawZone"
bucket_trusted = "bkt-musicstream-bi/Files/TrustedZone"
bucket_refined = "bkt-musicstream-bi/Files/RefinedZone"
bucket_metadata = "bkt-musicstream-bi/Files/09Metadata"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_music_info')
music_info_00=sqlContext.read.format("parquet").load(path_lake)
# orders_00.registerTempTable("orders_00")
# orders_00.cache()
# orders_00.count()
path_lake = os.path.join('s3://',bucket_trusted,'trusted_user_info')
user_info_00=sqlContext.read.format("parquet").load(path_lake)
# people_00.registerTempTable("people_00")
# people_00.cache()
# people_00.count()
path_lake = os.path.join('s3://',bucket_trusted,'trusted_user_listening_history')
user_listening_history_00=sqlContext.read.format("parquet").load(path_lake)
# products_00.registerTempTable("products_00")
# products_00.cache()
# products_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
music_info_00.dtypes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('ts_proc', 'bigint'), ('track_id', 'string'), ('name', 'string'), ('artist', 'string'), ('spotify_preview_url', 'string'), ('spotify_id', 'string'), ('tags', 'string'), ('year', 'int'), ('duration_ms', 'int'), ('danceability', 'double'), ('energy', 'double'), ('key', 'int'), ('loudness', 'double'), ('mode', 'int'), ('speechiness', 'double'), ('acousticness', 'double'), ('instrumentalness', 'double'), ('liveness', 'double'), ('valence', 'double'), ('tempo', 'double'), ('time_signature', 'int'), ('genre', 'string'), ('ts_proc_partition', 'bigint')]

In [11]:
metadados_fisico = []

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
for i in music_info_00.dtypes:
    tupla_final = i + ('music_info','trusted',)
    metadados_fisico.append(tupla_final)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
for i in user_info_00.dtypes:
    tupla_final = i + ('user_info','trusted',)
    metadados_fisico.append(tupla_final)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
for i in user_listening_history_00.dtypes:
    tupla_final = i + ('user_listening_history','trusted',)
    metadados_fisico.append(tupla_final)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
for i in metadados_fisico:
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('ts_proc', 'bigint', 'music_info', 'trusted')
('track_id', 'string', 'music_info', 'trusted')
('name', 'string', 'music_info', 'trusted')
('artist', 'string', 'music_info', 'trusted')
('spotify_preview_url', 'string', 'music_info', 'trusted')
('spotify_id', 'string', 'music_info', 'trusted')
('tags', 'string', 'music_info', 'trusted')
('year', 'int', 'music_info', 'trusted')
('duration_ms', 'int', 'music_info', 'trusted')
('danceability', 'double', 'music_info', 'trusted')
('energy', 'double', 'music_info', 'trusted')
('key', 'int', 'music_info', 'trusted')
('loudness', 'double', 'music_info', 'trusted')
('mode', 'int', 'music_info', 'trusted')
('speechiness', 'double', 'music_info', 'trusted')
('acousticness', 'double', 'music_info', 'trusted')
('instrumentalness', 'double', 'music_info', 'trusted')
('liveness', 'double', 'music_info', 'trusted')
('valence', 'double', 'music_info', 'trusted')
('tempo', 'double', 'music_info', 'trusted')
('time_signature', 'int', 'music_info', 'truste

In [16]:
schema = StructType([
    StructField("column", StringType(), nullable=False),
    StructField("type", StringType(), nullable=False),
    StructField("table", StringType(), nullable=False),
    StructField("zone", StringType(), nullable=False)
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
metadata_structure = spark.createDataFrame(data=metadados_fisico, schema=schema)
metadata_structure.registerTempTable('metadata_structure')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
metadata_structure.show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+--------------------+-------+
|             column|  type|               table|   zone|
+-------------------+------+--------------------+-------+
|            ts_proc|bigint|          music_info|trusted|
|           track_id|string|          music_info|trusted|
|               name|string|          music_info|trusted|
|             artist|string|          music_info|trusted|
|spotify_preview_url|string|          music_info|trusted|
|         spotify_id|string|          music_info|trusted|
|               tags|string|          music_info|trusted|
|               year|   int|          music_info|trusted|
|        duration_ms|   int|          music_info|trusted|
|       danceability|double|          music_info|trusted|
|             energy|double|          music_info|trusted|
|                key|   int|          music_info|trusted|
|           loudness|double|          music_info|trusted|
|               mode|   int|          music_info|trusted|
|        speec

In [19]:
path_meta = 's3://bkt-musicstream-bi/Files/09DescriptionFolder/music_info_desc.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
meta_desc = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ';').load(
       path_meta)
meta_desc.registerTempTable("meta_desc")
meta_desc.cache()
meta_desc.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20

In [21]:
#create new column with table name and music_info_dim in rows
#reorder columns as well
meta_desc = meta_desc.withColumn("table", lit("music_info"))
meta_desc_desired_order = ["table", "column", "description"]
meta_desc = meta_desc.select(meta_desc_desired_order)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
path_meta = 's3://bkt-musicstream-bi/Files/09DescriptionFolder/user_id_info_desc.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
meta_user_id = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ';').load(
       path_meta)
meta_desc.registerTempTable("meta_user_id")
meta_desc.cache()
meta_desc.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20

In [24]:
#create new column with table name and music_info_dim in rows
#reorder columns as well
#rename column desc to description
meta_user_id = meta_user_id.withColumn("table", lit("user_info"))
meta_desc_desired_order = ["table", "column", "desc"]
meta_user_id = meta_user_id.select(meta_desc_desired_order)
meta_user_id = meta_user_id.withColumnRenamed("desc", "description")
meta_user_id.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+----------------------+
|table    |column   |description           |
+---------+---------+----------------------+
|user_info|track_id |unique_id_track       |
|user_info|user_id  |unique_id_user        |
|user_info|playcount|number of times played|
+---------+---------+----------------------+

In [25]:
path_meta = 's3://bkt-musicstream-bi/Files/09DescriptionFolder/user_listening_history_desc.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
meta_user_listening_history = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ';').load(
       path_meta)
meta_user_listening_history.registerTempTable("meta_user_listening_history")
meta_user_listening_history.cache()
meta_user_listening_history.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

In [27]:
#create new column with table name and music_info_dim in rows
#reorder columns as well
#rename column desc to description
meta_user_listening_history = meta_user_listening_history.withColumn("table", lit("user_listening_history"))
meta_desc_desired_order = ["table", "column", "desc"]
meta_user_listening_history = meta_user_listening_history.select(meta_desc_desired_order)
meta_user_listening_history = meta_user_listening_history.withColumnRenamed("desc", "description")
meta_user_listening_history.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+---------+-------------+
|table                 |column   |description  |
+----------------------+---------+-------------+
|user_listening_history|track_id |unique_value,|
|user_listening_history|user_id  |unique_value,|
|user_listening_history|playcount|register_play|
+----------------------+---------+-------------+

In [28]:
# Append (concatenate) the three DataFrames vertically
complete_metadata = meta_desc.union(meta_user_id).union(meta_user_listening_history)

# Remove commas (",") from the "description" column
complete_metadata = complete_metadata.withColumn("description", regexp_replace("description", ",", ""))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
complete_metadata.show(2, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------------+
|table     |column  |description |
+----------+--------+------------+
|music_info|track_id|unique_value|
|music_info|artist  |artist_name |
+----------+--------+------------+
only showing top 2 rows

In [30]:
complete_metadata.registerTempTable('complete_metadata')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
metadata_structure.show(100, truncate=False)
complete_metadata.show(100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+----------------------+-------+
|column             |type  |table                 |zone   |
+-------------------+------+----------------------+-------+
|ts_proc            |bigint|music_info            |trusted|
|track_id           |string|music_info            |trusted|
|name               |string|music_info            |trusted|
|artist             |string|music_info            |trusted|
|spotify_preview_url|string|music_info            |trusted|
|spotify_id         |string|music_info            |trusted|
|tags               |string|music_info            |trusted|
|year               |int   |music_info            |trusted|
|duration_ms        |int   |music_info            |trusted|
|danceability       |double|music_info            |trusted|
|energy             |double|music_info            |trusted|
|key                |int   |music_info            |trusted|
|loudness           |double|music_info            |trusted|
|mode               |int   |music_info  

In [32]:
meta_desc_01 = spark.sql( 
    """
    select 
            t1.zone,
            t1.table,
            t1.column,
            t1.type,
            t2.description
            
    from
        metadata_structure as t1
        left join complete_metadata as t2
        on (t1.table = t2.table and t1.column = t2.column)
    """)
meta_desc_01.registerTempTable("meta_desc_01")
meta_desc_01.cache()
meta_desc_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

31

In [36]:
meta_desc_01.show(100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------+-------------------+------+----------------------------------------+
|zone   |table                 |column             |type  |description                             |
+-------+----------------------+-------------------+------+----------------------------------------+
|trusted|music_info            |ts_proc            |bigint|null                                    |
|trusted|music_info            |track_id           |string|unique_value                            |
|trusted|music_info            |name               |string|null                                    |
|trusted|music_info            |artist             |string|artist_name                             |
|trusted|music_info            |spotify_preview_url|string|access_to_song_demo                     |
|trusted|music_info            |spotify_id         |string|unique_value                            |
|trusted|music_info            |tags               |string|genres_sep_by_comma             

In [34]:
meta_desc_01

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[zone: string, table: string, column: string, type: string, description: string]

In [38]:
meta_desc_01.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['zone', 'table', 'column', 'type', 'description']

In [39]:
meta_desc_01.show(
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-------------------+------+--------------------+
|   zone|     table|             column|  type|         description|
+-------+----------+-------------------+------+--------------------+
|trusted|music_info|            ts_proc|bigint|                null|
|trusted|music_info|           track_id|string|        unique_value|
|trusted|music_info|               name|string|                null|
|trusted|music_info|             artist|string|         artist_name|
|trusted|music_info|spotify_preview_url|string| access_to_song_demo|
|trusted|music_info|         spotify_id|string|        unique_value|
|trusted|music_info|               tags|string| genres_sep_by_comma|
|trusted|music_info|               year|   int|        year_release|
|trusted|music_info|        duration_ms|   int|duration_of_song_...|
|trusted|music_info|       danceability|double| parameter_of_a_song|
|trusted|music_info|             energy|double| parameter_of_a_song|
|trusted|music_info|              

In [49]:
meta_desc_01 = meta_desc_01.withColumnRenamed("table", "name_table")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# Save File
output_lake = "tabela_metadados"
path_metadata = os.path.join('s3://', bucket_metadata, output_lake)
print(path_metadata)

meta_desc_01.coalesce(1).write \
    .parquet(path_metadata, mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/09Metadata/tabela_metadados